In [4]:
import os
import re

import pandas as pd
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [111]:
# nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/ulyumdzhi/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [11]:
from dotenv import load_dotenv
import lyricsgenius as lg

load_dotenv()
API_TOKEN = os.getenv('GEN_TOKEN')

def name_checker(name):
    # if name not in names:
    try:
        genius = lg.Genius(API_TOKEN,skip_non_songs=True, remove_section_headers=True)
        response = (genius.search_artist(name, max_songs=1, sort='popularity'))
        true_name = response.name
        return true_name  
    except:
        return name
    # return name

In [27]:
def clean_lemm_general(text_general):
    
    def clean(text):
        import re
        text_clean_pre = text.lower() # приводим все символы к нижнему регистру
        text_clean_pre = re.sub(r'\d+',' ', text_clean_pre) # удаляем все числа 
        text_clean_pre = re.sub(r'[^\w\s]','', text_clean_pre) # удаляем все знаки препинания
        text_clean_pre = re.sub(r'http\S+', ' ', text_clean_pre)
        text_clean_pre = re.sub(r'@\w+',' ',text_clean_pre)
        text_clean_pre = re.sub(r'#\w+', ' ', text_clean_pre)
        return text_clean_pre
    
    def nltk_stop_words(tokenized_text):
        clean_stop_words = [] 
        for element in enumerate(tokenized_text):
            clean_stop_words.append(' '.join([word for word in element if word not in sw]))
        return clean_stop_words
    
    text_clean_gen = [clean(i) for i in text_general.split()]
    text_lem = WordNetLemmatizer()
    lem_text = []
    for words in text_clean_gen:
        lem_text.append(' '.join([text_lem.lemmatize(word) for word in words.split()]))
    
    
    return ' '.join(lem_text)

In [28]:
text_dict = {}
for i in os.listdir('../data'):
    try:
        with open(f'../data/{i}', encoding='utf-8', newline='') as f:
            artist_text = f.read()
            text_dict[i[:-4]] = clean_lemm_general(artist_text)
    except:
        print(i)  
        
len(text_dict)

.DS_Store


In [104]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

tfidf = TfidfVectorizer(ngram_range=(1, 4))
tfidf_representation = tfidf.fit_transform(text_dict.values())
artists_similarity = 1 - pairwise_distances(tfidf_representation, metric="cosine")

In [112]:
dict_of_artist_similarity = dict(zip(text_dict.keys(), artists_similarity))

In [113]:
df = pd.DataFrame(dict_of_artist_similarity, index=text_dict.keys())

In [110]:
df['grandson'].sort_values(ascending=False).head(10)[1:]

Eminem                  0.244019
NEFFEX                  0.173702
Hollywood Undead        0.172948
Nickelback              0.167084
STARSET                 0.165498
Disturbed               0.154073
Flo Rida                0.148894
Bring Me The Horizon    0.147752
Mike Shinoda            0.144651
Name: grandson, dtype: float64